In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import urllib.request
import os
from PIL import Image
import io
import re 
import time
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import warnings
warnings.filterwarnings("ignore")

# . Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.


In [2]:
driver=webdriver.Chrome('chromedriver.exe')
url=("https://www.amazon.in/")
driver.get(url)
driver.maximize_window()
time.sleep(2)

In [3]:
search_product=input("Enter the Product Name: ")

Enter the Product Name: guitar


In [4]:
print("Product to search is: ",search_product)

Product to search is:  guitar


In [5]:
product=driver.find_element(By.ID,"twotabsearchtextbox").send_keys(search_product)
search_product=driver.find_element(By.ID,"nav-search-submit-button").click()

# . In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [6]:
# scrape all product urls
product_urls = []
start=0
end=3
for page in range(start,end):                                                 
    url=driver.find_elements(By.XPATH,'//a[@class = "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href"))                        
    nxt_button=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')     #scraping the list of buttons from the page
    nxt_button.click()
    time.sleep(2)

In [7]:
len(product_urls)

165

In [8]:
product = []
price = []
availability=[]
Brand=[]
ReturnExchange=[]
Delivery=[]
Ratings=[]


for url in product_urls[:10]:                                                        
    driver.get(url)
    time.sleep(2)
    
    try:
        brand= driver.find_element(By.XPATH,'//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span')
        Brand.append(brand.text)
    except NoSuchElementException:
        Brand.append('-') 
        
    try:
        exchange = driver.find_element(By.XPATH,'//*[@id="RETURNS_POLICY"]/span/div[2]/a')
        ReturnExchange.append(exchange.text)
    except NoSuchElementException:
        ReturnExchange.append('-') 
        
    try:
        delivery= driver.find_element(By.XPATH,'//span[@class="a-text-bold"]')
        Delivery.append(delivery.text)
    except NoSuchElementException:
        Delivery.append('-') 
        
    try:
        product.append(driver.find_element(By.XPATH,'//span[@id="productTitle"]').text)
    except:
        product.append('-')
      
    try:
        try:
            price.append(driver.find_element(By.XPATH,'//td[@class="a-span12"]').text)
        except:
            price.append(driver.find_element(By.CSS_SELECTOR,'span[class="a-price-whole"]').text)             
    except:
        price.append('-')
    
    try:
        availability.append(driver.find_element(By.XPATH,'//div[@id="availability"]').text)
    except:
        availability.append('-')
        
    try:
        Ratings.append(driver.find_element(By.XPATH,'//span[@class="a-size-medium"]').text)
    except:
        Ratings.append('-')

In [9]:
print(len(Brand),len(ReturnExchange),len(Delivery),len(product),len(price),len(availability),len(Ratings))

10 10 10 10 10 10 10


In [10]:
df=pd.DataFrame({'Brand':Brand,'ReturnExchange':ReturnExchange,'Price':price,'Delivery_by':Delivery,'urls':url,'Ratings':Ratings,'Name of Product':product,'Availability of product':availability})
df

,Brand,ReturnExchange,Price,Delivery_by,urls,Ratings,Name of Product,Availability of product
0,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
1,-,-,-,-,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
2,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
3,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
4,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
5,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
6,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
7,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
8,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-
9,-,-,-,,https://www.amazon.in/Intern-38-inch-Cutaway-D...,-,-,-


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

In [11]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf"]')    
search_bar.send_keys("fruits")

In [13]:
search_button = driver.find_element(By.XPATH,'//div[@class="zgAlFc"]')
search_button.click()

In [14]:
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\muham\Downloads\aslam"+str(i)+".jpg", "wb")
    file.write(response.content)

In [15]:
driver.close()

In [16]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf"]')    
search_bar.send_keys("cars") 

In [18]:
search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')   
search_button.click()

In [20]:
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
img = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in img:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\muham\Downloads\aslam"+str(i)+".jpg", "wb")
    file.write(response.content)

In [21]:
driver.close()

In [22]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf"]')    
search_bar.send_keys("Machine Learning") 

In [24]:
search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')   
search_button.click()

In [25]:
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
img = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in img:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\muham\Downloads\aslam"+str(i)+".jpg", "wb")
    file.write(response.content)

In [26]:
driver.close()

In [27]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf"]')    
search_bar.send_keys("Guitar") 

In [29]:
search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')   
search_button.click()

In [30]:
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
img = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in img:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\muham\Downloads\aslam"+str(i)+".jpg", "wb")
    file.write(response.content)

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) onwww.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [31]:
driver4=webdriver.Chrome('chromedriver.exe')
driver4.get("https://www.flipkart.com/")
time.sleep(2)
driver4.maximize_window()
pop_close = driver4.find_element("xpath","/html/body/div[2]/div/div/button").click()
product = input(" Enter the name of Smartphone that has to be searched : ")
search_product4=driver4.find_element(By.CLASS_NAME,'_3704LK').send_keys(product)

 Enter the name of Smartphone that has to be searched : one plus nord


In [32]:
search_button4= driver4.find_element(By.CLASS_NAME,'L0Z3Pu').click()

In [33]:
# Fetching urls of phones coming on 1st page
page1_urls = []
urls = driver4.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [34]:
#  creating  empty  list
Brand = []
Phone_name = []
Colour = []
RAM= []
ROM = []
Primary_Camera= []
Secondary_Camera = []
Display_Size= []
Battery_Capacity = []
Price = []
URL = []

In [35]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver4.get(url)              
    URL.append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver4.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
        
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver4.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Brand.append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Brand.append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Phone_name.append(name_tags.text)
    except NoSuchElementException:
        Phone_name.append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Colour.append(color_tags.text)
    except NoSuchElementException:
        Colour.append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        RAM.append(ram_tags.text)
    except NoSuchElementException:
        RAM.append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        ROM.append(rom_tags.text)
    except NoSuchElementException:
        ROM.append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pri_tags.text)
    except NoSuchElementException:
        Primary_Camera.append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Secondary_Camera.append(sec_cam.text)
    except NoSuchElementException:
        Secondary_Camera.append('-')
        
     #Scraping Display size data 
    try:
        disp_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Displa Size" : raise NoSuchElementException
        disp_size = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Display_Size.append(disp_size.text)
    except NoSuchElementException:
        Display_Size.append('-')
        
        #Scraping battery capacity data
    try:
        if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Battery_Capacity.append(bat_capa.text)
    except NoSuchElementException:
        Battery_Capacity.append('-')        
        
    #Scraping Price data
    try:
        price_tags = driver4.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Price.append(price_tags.text)
    except NoSuchElementException:
        Price.append('-')

In [36]:
smartphone=pd.DataFrame({"brand_name":Brand[:24],"smartphone_name":Phone_name[:24],"product_colour":Colour[:24],"product_RAM":RAM[:24],
                         "product_ROM":ROM[:24],"Primary_Camera":Primary_Camera[:24],"Secondary_Camera":Secondary_Camera[:24],
                         "Display_size":Display_Size[:24],"Battery_Capacity":Battery_Capacity[:24],"Phone_Price":Price[:24],"Product_URL":URL[:24]})

smartphone

,brand_name,smartphone_name,product_colour,product_RAM,product_ROM,Primary_Camera,Secondary_Camera,Display_size,Battery_Capacity,Phone_Price,Product_URL
0,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,-,-,-,-,"₹19,968",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord CE 2 Lite 5G,Blue Tide,8 GB,128 GB,-,-,-,-,"₹21,995",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,989",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary Camera,-,-,-,"₹19,989",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,989",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,Nord 2T 5G,Jade Fog,8 GB,128 GB,-,-,-,-,"₹28,999",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
6,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,-,-,-,-,"₹18,500",https://www.flipkart.com/oneplus-nord-ce-2-lit...
7,OnePlus,Nord CE 2 5G,Bahama Blue,8 GB,128 GB,Primary Camera,-,-,-,"₹24,998",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
8,OnePlus,Nord CE 2 Lite 5G,Black Dusk,8 GB,128 GB,-,-,-,-,"₹24,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
9,OnePlus,Nord 2T 5G,Gray Shadow,8 GB,128 GB,-,-,-,-,"₹28,900",https://www.flipkart.com/oneplus-nord-2t-5g-gr...


# 5.  Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [37]:
driver5=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening google maps web page
driver5.get("https://www.google.co.in/maps")
driver5.maximize_window()
time.sleep(2)

#Sending keyword for seach box and search button
city = input('Enter City name to search : ')
search_city = driver5.find_element(By.ID,"searchboxinput")                       
search_city.clear()                                                             
time.sleep(2)

search_city.send_keys(city)                                                     
search_button = driver5.find_element(By.ID,"searchbox-searchbutton").click()                                                             
time.sleep(3)
Location=driver5.current_url

try:
    coordinates=re.findall(r'@(.*)data',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
except IndexError as e:
    coordinates=re.findall(r'@(.*)z',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
print("\nlatitude={}".format(lat))
print("\nlongitude={}".format(lon))

Enter City name to search : delhi

latitude=28.6437954

longitude=76.8130369


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [40]:
driver6=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)
# opening Track.in web page
driver6.get("https://trak.in/")
driver6.maximize_window()
time.sleep(2)

In [41]:
#click on funding deals and go to that page
fund_btn=driver6.find_element(By.XPATH, '//li[@id="menu-item-1237902"]/a').get_attribute('href')
driver6.get(fund_btn)

In [43]:
date=[]
name=[]
industry=[]
subvertical=[]
location=[]
investors_name=[]
investors_type=[]
amount=[]

In [44]:
dates=driver.find_elements(By.XPATH,'//td[@class="column-2"]')
for i in dates[5:29]:
       date.append(i.text)
names=driver.find_elements(By.XPATH,'//td[@class="column-3"]')
for i in names[5:29]:
       name.append(i.text)
industrys=driver.find_elements(By.XPATH,'//td[@class="column-4"]')
for i in industrys[5:29]:
       industry.append(i.text)
subverticals=driver.find_elements(By.XPATH,'//td[@class="column-5"]')
for i in subverticals[5:29]:
       subvertical.append(i.text)
locations=driver.find_elements(By.XPATH,'//td[@class="column-6"]')
for i in locations[5:29]:
       location.append(i.text)
investors_names=driver.find_elements(By.XPATH,'//td[@class="column-7"]')
for i in investors_names[5:29]:
       investors_name.append(i.text)
investors_types=driver.find_elements(By.XPATH,'//td[@class="column-8"]')
for i in investors_types[5:29]:
       investors_type.append(i.text)
amounts=driver.find_elements(By.XPATH,'//td[@class="column-9"]')
for i in amounts[5:29]:
       amount.append(i.text)

In [45]:
print(len(date),len(name),len(industry),len(subvertical),len(location),len(investors_name),len(investors_type),len(amount))


0 0 0 0 0 0 0 0


In [46]:
df=pd.DataFrame({'Date':date,'Name':name,'Industry':industry,'Subvertical':subvertical,'Location':location,'Investors_name':investors_name,'Investors_type':investors_type,'Amount':amount})
df

,Date,Name,Industry,Subvertical,Location,Investors_name,Investors_type,Amount


#  7. Write a program to scrap all the available details of best gaming laptops from digit.in.


In [47]:
driver7=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening Digit.in web page
driver7.get("https://digit.in/")
driver7.maximize_window()
time.sleep(2)

In [48]:
driver7.find_element(By.XPATH,"/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00856643]
	(No symbol) [0x007EBE21]
	(No symbol) [0x006EDA9D]
	(No symbol) [0x00721342]
	(No symbol) [0x0072147B]
	(No symbol) [0x00758DC2]
	(No symbol) [0x0073FDC4]
	(No symbol) [0x00756B09]
	(No symbol) [0x0073FB76]
	(No symbol) [0x007149C1]
	(No symbol) [0x00715E5D]
	GetHandleVerifier [0x00ACA142+2497106]
	GetHandleVerifier [0x00AF85D3+2686691]
	GetHandleVerifier [0x00AFBB9C+2700460]
	GetHandleVerifier [0x00903B10+635936]
	(No symbol) [0x007F4A1F]
	(No symbol) [0x007FA418]
	(No symbol) [0x007FA505]
	(No symbol) [0x0080508B]
	BaseThreadInitThunk [0x76666739+25]
	RtlGetFullPathName_UEx [0x77968FD2+1218]
	RtlGetFullPathName_UEx [0x77968F9D+1165]


In [49]:
product_name=[]
seller=[]
price=[]

product_tag=driver7.find_elements(By.XPATH,"//*[@id='summtable']/tbody/tr/td[1]")
for i in product_tag:
    try:
        product_name.append(i.text)
    except NoSuchElementException:
        product_name.append("-")

seller_tag=driver7.find_elements(By.CLASS_NAME,"smmerchant")
for i in seller_tag:
    try:
        seller.append(i.text)
    except NoSuchElementException:
        seller.append("-")

price_tag=driver7.find_elements(By.CLASS_NAME,"smprice")
for i in price_tag:
    try:
        price.append(i.text)
    except NoSuchElementException:
        price.append("-")

Gaming=pd.DataFrame({"Product Name":product_name,"Seller":seller,"Price":price})
Gaming

,Product Name,Seller,Price


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [50]:
driver8=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening web page
driver8.get("https://www.forbes.com/billionaires/")
driver8.maximize_window()
time.sleep(2)

In [51]:
# Create Empty list
rank = []
name = []
net_worth = []
age = []
citzn = []
source = []
industry = []

#Fetching Rank
rank_tag = driver8.find_elements(By.XPATH,"//div[@class='rank']")
for r in rank_tag:
    try:
        rank.append(r.text)
    except NoSuchElementException:
        rank.append("-")
    
#Fetching Name
name_tag = driver8.find_elements(By.XPATH,"//div[@class='personName']")
for n in name_tag:
    try:
        name.append(n.text)
    except NoSuchElementException:
        name.append("-")
        
#Fetching Net Worth
netwrth_tag = driver8.find_elements(By.XPATH,"//div[@class='netWorth']")
for nt in netwrth_tag:
    try:
        net_worth.append(nt.text)
    except NoSuchElementException:
        net_worth.append("-")
        
#Fetching Age
age_tag = driver8.find_elements(By.XPATH,"//div[@class='age']")
for a in age_tag:
    try:
        age.append(a.text)
    except NoSuchElementException:
        age.append("-")
    
#Fetching Citizenship
cit_tag = driver8.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
for c in cit_tag:
    try:
        citzn.append(c.text)
    except NoSuchElementException:
        citzn.append("-")
    
#Fetching Source
src_tag = driver8.find_elements(By.XPATH,"//span[@class='source-text']")
for s in src_tag:
    try:
        source.append(s.text)
    except NoSuchElementException:
        source.append("-")
    
#Fetching Industry
ind_tag = driver8.find_elements(By.XPATH,"//div[@class='category']")
for ind in ind_tag:
        try:
            industry.append(ind.text)
        except NoSuchElementException:
            industry.append("-")

In [52]:
# Length of all the coloumns
len(rank),len(name),len(net_worth),len(age),len(citzn),len(source),len(industry)

(200, 200, 200, 200, 200, 200, 200)

In [53]:
#create Dataframe
Billionaires =pd.DataFrame({'Rank':rank,'Name':name,'Net Worth':net_worth,'Age': age,'Citizenship/Country':citzn,'Source':source,'Industry':industry})
Billionaires


,Rank,Name,Net Worth,Age,Citizenship/Country,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.


In [54]:
driver9=webdriver.Chrome('chromedriver.exe')
driver9.get("https://www.youtube.com/watch?v=USccSZnS8MQ")
driver9.maximize_window()
time.sleep(2)

In [57]:
i=0
while(i<100):
    driver9.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver9.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1
    
comment = []
upvote = []
comment_time = []

comment_tag=(driver9.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]'))
for i in comment_tag:             
    try:
        comment.append(i.text)
    except NoSuchElementException:
        comment.append("-")

upvote_tag=(driver9.find_elements(By.XPATH,"//*[@id='vote-count-middle']"))
for i in upvote_tag:             
    try:
        upvote.append(i.text)
    except NoSuchElementException:
        upvote.append("-")

comment_time_tag=(driver9.find_elements(By.XPATH,"//*[@id='header-author']/yt-formatted-string/a"))
for i in comment_time_tag:             
    try:
        comment_time.append(i.text)
    except NoSuchElementException:
        comment_time.append("-")

In [58]:
video=pd.DataFrame({"Comment":comment,"Upvote":upvote,"Comment_ Time":comment_time})
video


ValueError: All arrays must be of the same length

# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description

In [59]:
driver0=webdriver.Chrome('chromedriver.exe')
driver0.get("https://www.hostelworld.com/")
driver0.maximize_window()
time.sleep(2)

In [60]:
driver0.find_element(By.ID,'location-text-input-field').click()
time.sleep(2)
driver0.find_element(By.XPATH,'//input[@id="location-text-input-field"]').send_keys('London')
time.sleep(2)

In [61]:
driver0.find_element(By.XPATH,'//*[@id="predicted-search-results"]/li[1]').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="predicted-search-results"]/li[1]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00856643]
	(No symbol) [0x007EBE21]
	(No symbol) [0x006EDA9D]
	(No symbol) [0x00721342]
	(No symbol) [0x0072147B]
	(No symbol) [0x00758DC2]
	(No symbol) [0x0073FDC4]
	(No symbol) [0x00756B09]
	(No symbol) [0x0073FB76]
	(No symbol) [0x007149C1]
	(No symbol) [0x00715E5D]
	GetHandleVerifier [0x00ACA142+2497106]
	GetHandleVerifier [0x00AF85D3+2686691]
	GetHandleVerifier [0x00AFBB9C+2700460]
	GetHandleVerifier [0x00903B10+635936]
	(No symbol) [0x007F4A1F]
	(No symbol) [0x007FA418]
	(No symbol) [0x007FA505]
	(No symbol) [0x0080508B]
	BaseThreadInitThunk [0x76666739+25]
	RtlGetFullPathName_UEx [0x77968FD2+1218]
	RtlGetFullPathName_UEx [0x77968F9D+1165]


In [62]:
driver0.find_element("xpath",'//div[@class="search-button"]').click()

In [63]:
# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

In [64]:
while(True):
    
    # Hostel name
    names = driver0.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city
    distance = driver0.find_elements(By.XPATH,'//span[@class="description"]')
    for d in distance:
        Distance.append(d.text)
    time.sleep(2)
        
    #Review    
    review = driver0.find_elements(By.XPATH,'//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total number of reviews     
    t_review = driver0.find_elements(By.XPATH,'//div[@class="reviews"]')
    for t in t_review:
        total_reviews.append(t.text)
    time.sleep(2)
    
    # facilities
    service = driver0.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver0.find_elements(By.XPATH,'//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
    try:
        next_button = driver0.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(2) 

# Separate  Privates_From price  and  Dorms_From price
private = []
for i in range(0,len(price),2):
    private.append(price[i])
time.sleep(2)

dorms = []
for i in range(1,len(price),2):
    dorms.append(price[i]) 

In [ ]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver0.find_elements(By.XPATH,'//h2[@class="title title-6"]/a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver0.find_element(By.XPATH,'//div[@class="pagination-item pagination-prev"]')
        next_button.click()
    except:
        break
        
Rate = []
for page in hostel_url:
    driver0.get(page)
    
    # Rating
    try:
        ratings = driver0.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    # Property Description
    try:
        pd = driver0.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 


In [65]:
print(
    len(Hostel_Name),len(Distance),len(Rating),len(total_reviews),len(overall_review),
    len(private),len(dorms),len(facilities),len(property_description))

0 0 0 0 0 0 0 0 0


In [66]:
Hostel=pd.DataFrame({"Hostel Name":Hostel_Name,"Distance":Distance,"Rating":Rating,"Total Reviews":total_reviews,"Overall Review":overall_review[:30],
                     "Private Price":private,"Dorms Price":dorms,"Facilities":facilities,"Property Description":property_description})
Hostel

,Hostel Name,Distance,Rating,Total Reviews,Overall Review,Private Price,Dorms Price,Facilities,Property Description
